# Pipeline Monitoring Dashboard
This notebook provides queries to monitor the Kafka Alerting Pipeline health and performance.


In [ ]:
# Configuration - Update with your catalog and schema
CATALOG = "andrea_tardif"
SCHEMA = "kafka_pipeline_dev"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")


## 1. Overall Pipeline Health


In [ ]:
# Check recent errors
spark.sql(f"""
SELECT 
    client_id,
    table_name,
    COUNT(*) as error_count,
    MAX(error_timestamp) as last_error,
    COLLECT_LIST(error_message) as error_messages
FROM {CATALOG}.{SCHEMA}.pipeline_errors
WHERE error_timestamp >= current_timestamp() - INTERVAL 1 HOUR
GROUP BY client_id, table_name
ORDER BY error_count DESC
""").display()


## 2. Data Volume by Client


In [ ]:
# Volume by client in the last hour
spark.sql(f"""
SELECT 
    client_id,
    client_name,
    COUNT(*) as message_count,
    MIN(ingestion_timestamp) as first_message,
    MAX(ingestion_timestamp) as last_message
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
WHERE ingestion_timestamp >= current_timestamp() - INTERVAL 1 HOUR
GROUP BY client_id, client_name
ORDER BY message_count DESC
""").display()


## 3. Data Quality Metrics


In [ ]:
# Compare record counts across layers (for client_001 as example)
from pyspark.sql.functions import lit

try:
    bronze_count = spark.table(f"{CATALOG}.{SCHEMA}.bronze_client_001").count()
    silver_count = spark.table(f"{CATALOG}.{SCHEMA}.silver_client_001").count()
    gold_count = spark.table(f"{CATALOG}.{SCHEMA}.gold_client_001_summary").count()
    
    quality_df = spark.createDataFrame([
        ("Bronze", bronze_count),
        ("Silver", silver_count),
        ("Gold", gold_count)
    ], ["Layer", "Record Count"])
    
    quality_df.display()
    
    # Calculate drop rates
    bronze_to_silver_drop = ((bronze_count - silver_count) / bronze_count * 100) if bronze_count > 0 else 0
    print(f"\nDrop rate from Bronze to Silver: {bronze_to_silver_drop:.2f}%")
except Exception as e:
    print(f"Error calculating quality metrics: {str(e)}")
    print("Ensure tables exist and have data")


## 4. Event Type Distribution


In [ ]:
# Event type distribution by client
spark.sql(f"""
SELECT 
    client_id,
    event_type,
    COUNT(*) as event_count,
    ROUND(AVG(amount), 2) as avg_amount,
    ROUND(SUM(amount), 2) as total_amount
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
WHERE ingestion_timestamp >= current_timestamp() - INTERVAL 1 HOUR
GROUP BY client_id, event_type
ORDER BY client_id, event_count DESC
""").display()


## 5. Gold Layer Metrics


In [ ]:
# View latest gold metrics for client_001
try:
    spark.sql(f"""
    SELECT 
        client_id,
        window_start,
        window_end,
        event_type,
        status,
        event_count,
        ROUND(total_amount, 2) as total_amount,
        ROUND(avg_amount, 2) as avg_amount,
        min_amount,
        max_amount
    FROM {CATALOG}.{SCHEMA}.gold_client_001_summary
    ORDER BY window_start DESC
    LIMIT 20
    """).display()
except Exception as e:
    print(f"Error: {str(e)}")
    print("Gold table may not exist yet or may not have data")


## 6. Kafka Offset Monitoring


In [ ]:
# Track Kafka offset progress
spark.sql(f"""
SELECT 
    kafka_partition,
    MIN(kafka_offset) as min_offset,
    MAX(kafka_offset) as max_offset,
    COUNT(*) as messages_processed,
    MAX(ingestion_timestamp) as last_ingestion_time
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
GROUP BY kafka_partition
ORDER BY kafka_partition
""").display()


## 7. Client Status Summary


In [ ]:
# Status distribution by client
spark.sql(f"""
SELECT 
    client_id,
    status,
    COUNT(*) as status_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY client_id), 2) as percentage
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
WHERE ingestion_timestamp >= current_timestamp() - INTERVAL 1 HOUR
GROUP BY client_id, status
ORDER BY client_id, status_count DESC
""").display()


## 8. Historical Error Trends


In [ ]:
# Error trends over the last 24 hours
spark.sql(f"""
SELECT 
    DATE_TRUNC('hour', error_timestamp) as error_hour,
    client_id,
    COUNT(*) as error_count
FROM {CATALOG}.{SCHEMA}.pipeline_errors
WHERE error_timestamp >= current_timestamp() - INTERVAL 24 HOURS
GROUP BY DATE_TRUNC('hour', error_timestamp), client_id
ORDER BY error_hour DESC, error_count DESC
""").display()


## 9. Sample Data Inspection


In [ ]:
# View recent messages
spark.sql(f"""
SELECT 
    client_id,
    client_name,
    event_timestamp,
    event_type,
    amount,
    status,
    ingestion_timestamp
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
ORDER BY ingestion_timestamp DESC
LIMIT 20
""").display()


## 10. Custom Alert Query
Create custom alerts based on business rules


In [ ]:
# Example: Alert if any client has > 10 failed transactions in the last hour
alerts = spark.sql(f"""
SELECT 
    client_id,
    COUNT(*) as failed_count,
    MAX(event_timestamp) as last_failure
FROM {CATALOG}.{SCHEMA}.kafka_raw_bronze
WHERE status = 'failed'
  AND ingestion_timestamp >= current_timestamp() - INTERVAL 1 HOUR
GROUP BY client_id
HAVING COUNT(*) > 10
ORDER BY failed_count DESC
""")

if alerts.count() > 0:
    print("⚠️  ALERT: The following clients have high failure rates:")
    alerts.display()
else:
    print("✅ No alerts - all clients within acceptable failure thresholds")
